# Foundations of Computer Science - progetto 2022/23

- Mattia Birti [897092] 
- Alberto Porrini [826306]
- Gloria Longo [864579]

You have to work on the Dogs adoptions dataset:
*  [Dogs](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogs.csv)
*  [Dog Travel](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogTravel.csv)
*  [NST-EST2021-POP](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/NST-EST2021-POP.csv)

### Notes

1. It is mandatory to use GitHub for developing the project.
1. The project must be a jupyter notebook.
1. There is no restriction on the libraries that can be used, nor on the Python version.
1. All questions on the project must be asked in a public channel on Zulip.
1. At most 3 students can be in each group. You must create the groups by yourself.
1. You do not have to send me the project before the discussion.s

## Libraries

In [1]:
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher

### Import data from GitHub

#### Dogs.csv

In [2]:
dogs = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogs.csv", sep=",", encoding='latin-1')

In [3]:
dogs.shape

(58180, 37)

In [4]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


#### DogTravel.csv

In [5]:
dogTravel = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogTravel.csv", sep=",", encoding='latin-1')

In [6]:
dogTravel.shape

(6194, 9)

In [7]:
dogTravel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


#### NST-EST.csv

In [8]:
nstest = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/NST-EST2021-POP.csv", header=None, sep=",", encoding='latin-1')

In [9]:
nstest.shape

(51, 2)

In [10]:
nstest.head()

,0,1
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


## 1. Extract all dogs with status that is not adoptable

Explore the dog's column names

In [11]:
dogs.columns

Index(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs', 'shots_current',
       'env_children', 'env_dogs', 'env_cats', 'name', 'status', 'posted',
       'contact_city', 'contact_state', 'contact_zip', 'contact_country',
       'stateQ', 'accessed', 'type.y', 'description', 'stay_duration',
       'stay_cost'],
      dtype='object')

Explore the first row

In [12]:
dogs.iloc[0,:]

id                                                          46042150
org_id                                                         NV163
url                https://www.petfinder.com/dog/harley-46042150/...
type.x                                                           Dog
species                                                          Dog
breed_primary                         American Staffordshire Terrier
breed_secondary                                          Mixed Breed
breed_mixed                                                     True
breed_unknown                                                  False
color_primary                                          White / Cream
color_secondary                          Yellow / Tan / Blond / Fawn
color_tertiary                                                   NaN
age                                                           Senior
sex                                                             Male
size                              

We can see that there are two variables (posted, accessed) that contain dates, let's check if they are in the right format

In [13]:
type(dogs.loc[0, 'posted'])
type(dogs.loc[0, 'accessed'])

str

Since they are in string format, we convert them to date format

In [14]:
#for i in range(len(dogs)):
        #dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        #dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")

We can see that there are some variables that do not contain a date, let's see which ones

In [15]:
for i in range(len(dogs)):
    if(len(dogs.loc[i,'posted'])!=24):
        print(dogs.iloc[i,])

id                                                          41330726
org_id                                                         NV173
url                https://www.petfinder.com/dog/gunther-gunny-41...
type.x                                                           Dog
species                                                          Dog
breed_primary                                    German Shepherd Dog
breed_secondary                                                  NaN
breed_mixed                                                    False
breed_unknown                                                  False
color_primary                                                    NaN
color_secondary                                                  NaN
color_tertiary                                                   NaN
age                                                            Young
sex                                                             Male
size                              

The problem with these rows is that they have not been split properly in cell 'name'. Let's solve the problem and transform the dates into datatime format

In [16]:
for i in range(len(dogs)):
    if(len(dogs.loc[i,'posted'])==24):
        dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")
    else :
        j=26
        l=(dogs.iloc[i,24]).split(' ',1)
        prov=dogs.iloc[i,25]
        dogs.iloc[i,25]=l[1]
        dogs.iloc[i,24]=l[0]
        while(j<34):
            prov2=dogs.iloc[i,j]
            dogs.iloc[i,j]=prov
            prov=prov2
            j=j+1
        dogs.loc[i, 'posted']=datetime.strptime(dogs.loc[i,'posted'], "%Y-%m-%dT%H:%M:%S+0000")
        dogs.loc[i,'accessed']=datetime.strptime(dogs.loc[i,'accessed'], "%Y-%m-%d")

Now we can solve this task using a 'while' loop which scrolls through all the rows of the dataframe and stores only the identification code of the dogs that have a status other than adoptable

In [17]:
lista=[]
i=0
while(i<len(dogs)):
    if (dogs.loc[i,'status']!='adoptable'):
        lista.append(dogs.loc[i,'id'])
    i=i+1
lista

[41330726,
 38169117,
 45833989,
 45515547,
 45294115,
 45229004,
 45227052,
 45569380,
 44694387,
 36978896,
 33218331,
 42092005,
 39594038,
 45895274,
 45964719,
 44538917,
 41430442,
 45907639,
 45362806,
 32590894,
 31426754,
 46037827,
 44044071,
 27521132,
 38473806,
 34101432,
 45958435,
 45927580,
 45916348,
 45733027,
 45413997,
 45406516,
 45264615]

## 2. For each (primary) breed, determine the number of dogs

In [18]:
dogs.groupby('breed_primary').count()['id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

Let's see all the primary breed with a groupBy:

In [19]:
nBreed = dogs.groupby('breed_primary').count()[['species','breed_secondary']]
nBreed

,species,breed_secondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


As we can see we can use "species" as total number of individues for this species.
Affenpischer are in total 17 dogs, 2 of those are mixed breed.

Rename colums

In [20]:
nBreed = nBreed.rename( columns = {'species':'total', 'breed_secondary':'nSecondary'})
nBreed

,total,nSecondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


We have to compute how many are primary breed:

In [21]:
nBreed['nPrimary'] = nBreed['total'] - nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary
breed_primary,,,
Affenpinscher,17,2,15
Afghan Hound,4,1,3
Airedale Terrier,19,9,10
Akbash,3,0,3
Akita,181,52,129
...,...,...,...
Wirehaired Pointing Griffon,1,0,1
Wirehaired Terrier,60,18,42
Xoloitzcuintli / Mexican Hairless,11,4,7


It's time to compute the ratio between Primary and Secondary Breed.

In [22]:
nBreed['ratioBreed'] = nBreed['nPrimary'] / nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary,ratioBreed
breed_primary,,,,
Affenpinscher,17,2,15,7.500000
Afghan Hound,4,1,3,3.000000
Airedale Terrier,19,9,10,1.111111
Akbash,3,0,3,inf
Akita,181,52,129,2.480769
...,...,...,...,...
Wirehaired Pointing Griffon,1,0,1,inf
Wirehaired Terrier,60,18,42,2.333333
Xoloitzcuintli / Mexican Hairless,11,4,7,1.750000


As we can see, there are many 'inf' values. It is correct because for that breed there are not any secondary breed.

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

We use the group by method to solve this task starting from the latest posted timestamp.

In [23]:
df_new = dogs[dogs.groupby('breed_primary')['posted'].transform('max') == dogs['posted']]
df_new.loc[ :, ['breed_primary', 'posted']]

,breed_primary,posted
0,American Staffordshire Terrier,2019-09-20 16:37:59
6,Italian Greyhound,2019-09-20 06:42:30
749,American Bulldog,2019-09-20 16:06:25
1034,Pit Bull Terrier,2019-09-20 17:20:30
1035,Cocker Spaniel,2019-09-20 17:20:30
...,...,...
55810,Mountain Dog,2019-09-17 06:00:26
56127,Tosa Inu,2019-09-04 00:48:24
56696,Thai Ridgeback,2019-05-11 12:51:57
57459,Saint Bernard,2019-09-19 17:41:48


In [24]:
df_new = dogs[dogs.groupby('breed_primary')['posted'].transform('min') == dogs['posted']]
df_new.loc[ :, ['breed_primary', 'posted']]

,breed_primary,posted
693,Shiba Inu,2016-02-22 23:53:06
726,Siberian Husky,2011-01-14 21:58:52
728,Mastiff,2010-03-13 00:00:00
735,Boston Terrier,2009-03-23 00:00:00
738,Otterhound,2008-03-14 00:00:00
...,...,...
56902,American Hairless Terrier,2013-07-03 10:05:41
57449,Australian Shepherd,2008-11-06 00:00:00
57450,Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00
58090,Presa Canario,2016-10-08 18:20:52


## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

we create a table with count of male dogs for each state (contanct_state) 

In [25]:
dog_male=dogs[dogs['sex']=='Male']
dog_male=dog_male.groupby(['contact_state'])[['sex']].count()
dog_male.tail()

,sex
contact_state,
VT,276
WA,686
WI,277
WV,333
WY,31


we recreate the same table that we have done before with female dogs for each state

In [26]:
dog_female=dogs[dogs['sex']=='Female']
dog_female=dog_female.groupby(['contact_state'])[['sex']].count()
dog_female.tail()

,sex
contact_state,
VT,234
WA,598
WI,265
WV,232
WY,21


we create a new table call "dog_sex" thanks to merge between male dogs and female dogs on contact_state

In [27]:
dog_sex=pd.merge(dog_male,dog_female,on="contact_state")
dog_sex.rename( columns = {'sex_x':'tot_male','sex_y':'tot_female'})
dog_sex=dog_sex.rename( columns = {'sex_x':'tot_male','sex_y':'tot_female'})
dog_sex.tail()

,tot_male,tot_female
contact_state,,
VT,276,234
WA,686,598
WI,277,265
WV,333,232
WY,31,21


we compute the imbalance like difference between male and female dogs and after we calculate the absolute value of the imbalance given that we consider only positive value of imbalance for each contact_state

In [28]:
dog_sex["imbalance"]=dog_sex["tot_male"]-dog_sex["tot_female"]
dog_sex["imbalance"]=abs(dog_sex["imbalance"])
dog_sex.tail()

,tot_male,tot_female,imbalance
contact_state,,,
VT,276,234,42
WA,686,598,88
WI,277,265,12
WV,333,232,101
WY,31,21,10


we found the state with the largest imbalance

In [29]:
dog_sex[dog_sex["imbalance"]==dog_sex["imbalance"].max()]

,tot_male,tot_female,imbalance
contact_state,,,
OH,1439,1234,205


In [30]:
dog_sex["imbalance"].nlargest(1)

contact_state
OH    205
Name: imbalance, dtype: int64

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

Explore the dog's column names

In [31]:
dogs.columns

Index(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs', 'shots_current',
       'env_children', 'env_dogs', 'env_cats', 'name', 'status', 'posted',
       'contact_city', 'contact_state', 'contact_zip', 'contact_country',
       'stateQ', 'accessed', 'type.y', 'description', 'stay_duration',
       'stay_cost'],
      dtype='object')

Compute the mean of the stay duration and stay cost for the pair AGE, SIZE;

In [32]:
AgeSize = dogs.groupby(['age', 'size'])[['stay_duration','stay_cost']].mean()
AgeSize

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

Let's start grouping the dataset by the variable id

In [33]:
count=dogTravel.groupby('id').count()

We reset the index

In [34]:
count.reset_index(inplace=True)

Now we compute a list of only dogs that have made at least 2 trips

In [35]:
list_index=(count[count['index']>2])['id'].tolist()

Finally we build a dictionary having as key the identification code of the dog that has made at least 2 trips, and as value its primary breed. 
To do this a for loop will be used which compares each identification code of the dog in the staring dataset with those contained in the list created above



In [36]:
diz={}
for index in list_index:
    for i in range(len(dogs)):
        if (dogs.loc[i, 'id']==index):
            diz[dogs.loc[i, 'id']]= dogs.loc[i, 'breed_primary']
diz

{16657005: 'Pit Bull Terrier',
 20905974: 'Chow Chow',
 24894870: 'Hound',
 24894894: 'Hound',
 33218331: 'Alaskan Malamute',
 36978896: 'Alaskan Malamute',
 37108842: 'Pit Bull Terrier',
 37253070: 'Labrador Retriever',
 37848260: 'Labrador Retriever',
 38050885: 'Pit Bull Terrier',
 38495992: 'Pit Bull Terrier',
 38664932: 'Pit Bull Terrier',
 39608594: 'Pointer',
 40036107: 'Pit Bull Terrier',
 40103682: 'Rat Terrier',
 41144335: 'Chihuahua',
 41359772: 'German Shepherd Dog',
 42445043: 'Dutch Shepherd',
 42525486: 'Cairn Terrier',
 42778494: 'Jack Russell Terrier',
 42825396: 'Parson Russell Terrier',
 42835971: 'Pit Bull Terrier',
 43082511: 'Saluki',
 43379087: 'Labrador Retriever',
 43401994: 'Labrador Retriever',
 43529811: 'Mastiff',
 43576891: 'Labrador Retriever',
 43576898: 'Labrador Retriever',
 43618547: 'Alaskan Malamute',
 43679504: 'Pit Bull Terrier',
 43829075: 'American Bulldog',
 43956989: 'Mixed Breed',
 44070616: 'Alaskan Malamute',
 44077433: 'Labrador Retriever'

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

To resolve the question, we create two different tables, the first table is compose with only observation field manual full.

In [37]:
manual_not_na=dogTravel[dogTravel['manual'].notna()]
manual_not_na

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
6,6,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
7,7,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
330,330,45276595,Guntersville,AL,We call this little guy Bono. He is a happy a...,Albertville,Alabama,NaN,True
...,...,...,...,...,...,...,...,...,...
6183,6183,45017651,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True
6184,6184,44659739,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True
6185,6185,44289536,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True
6187,6187,42117845,Fairmont,WV,This is Dachshund Chihuahua Blue who weighs 7l...,WV,West Virginia,NaN,True


in this table we create a new column ("correct_state") with the same values in manual (cause are the values that will be overrides in found column)

In [38]:
manual_not_na["correct_state"] = manual_not_na["manual"]
manual_not_na

/var/folders/h7/63fpfjks70z8hgf9c6f28cl00000gn/T/ipykernel_2266/2206628724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual_not_na["correct_state"] = manual_not_na["manual"]


,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland
6,6,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN,Ohio
7,7,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN,Ohio
330,330,45276595,Guntersville,AL,We call this little guy Bono. He is a happy a...,Albertville,Alabama,NaN,True,Alabama
...,...,...,...,...,...,...,...,...,...,...
6183,6183,45017651,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True,West Virginia
6184,6184,44659739,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True,West Virginia
6185,6185,44289536,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,West Virginia,NaN,True,West Virginia
6187,6187,42117845,Fairmont,WV,This is Dachshund Chihuahua Blue who weighs 7l...,WV,West Virginia,NaN,True,West Virginia


on second table, we subset only the observations with empty manual values

In [39]:
manual_na=dogTravel[dogTravel['manual'].isna()]
manual_na

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
5,5,43082511,Manchester,CT,Brooke has an unusual past. She was rescued f...,Afghanistan,NaN,NaN,NaN
8,8,45987719,Locust Fork,AL,Meet Trixie... she is a female 2yr. Old Chihua...,Alabama,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6188,6188,41298157,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN


in this table we create a new column ("correct_state") with the same values in found (cause are the values that will be not overrides)

In [40]:
manual_na["correct_state"] = manual_na["found"]
manual_na

/var/folders/h7/63fpfjks70z8hgf9c6f28cl00000gn/T/ipykernel_2266/611043492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual_na["correct_state"] = manual_na["found"]


,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN,Arkansas
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN,Adaptil
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN,Afghanistan
5,5,43082511,Manchester,CT,Brooke has an unusual past. She was rescued f...,Afghanistan,NaN,NaN,NaN,Afghanistan
8,8,45987719,Locust Fork,AL,Meet Trixie... she is a female 2yr. Old Chihua...,Alabama,NaN,NaN,NaN,Alabama
...,...,...,...,...,...,...,...,...,...,...
6188,6188,41298157,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN,WV
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN,WV
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN,Wyoming
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN,Yazmin


subsequantly we recreate a new df with the concatenate of the two tables that we had created before, and we order them with index sort

In [41]:
df=pd.concat([manual_na,manual_not_na])
df=df.sort_index()
df

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN,Arkansas
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN,Adaptil
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN,Afghanistan
...,...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN,WV
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN,Wyoming
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN,Yazmin
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Young,Ohio,NaN,NaN,Ohio


now we have got a column correct_state complete with the values overrides and not overrides, so we replace that values at the column found of dogTravel dataframe and we ending the point

In [42]:
dogTravel["found"]=df["correct_state"]
dogTravel

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Ohio,Ohio,NaN,NaN


In the last result, we have got in found column the manual values only if they are not "NaN" 

## 9. For each state, compute the ratio between the number of travels and the population.

In [43]:
import pycountry #have to install it (pip install pycountry)

### First Dataset Analysis

In [44]:
dogTravel.shape #number of total travell done

(6194, 9)

In [45]:
dogTravel.groupby(['contact_state']).count().shape 
#number of countries that have made trips

(45, 8)

In [46]:
nstest.shape #number of country in csv population

(51, 2)

51 states, 45 states that have made trips, total trips 6149

In [47]:
#change label
nTravelForState = dogTravel.groupby(['contact_state']).count()[['index']].reset_index().rename(columns={'contact_state':'state', 'index':'nTravel'})
nTravelForState.head()

,state,nTravel
0,17325,10
1,AL,75
2,AR,10
3,AZ,70
4,CA,28


In [48]:
nTravelForState['nTravel'].sum() #sum is correct

6194

In [49]:
nTravelForState.shape

(45, 2)

### There is an error

In [50]:
dogTravel[dogTravel['contact_state'] == '17325']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
3237,3237,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN
3238,3238,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN
3714,3714,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN
3715,3715,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN
6029,6029,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN
6030,6030,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN
6074,6074,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Washington DC,NaN,True,NaN
6075,6075,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Washington DC,NaN,True,NaN
6133,6133,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,West Virginia,NaN,True,NaN
6134,6134,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",West Virginia,NaN,True,NaN


The number refers to state of Pensylvania. It needs to be corrected

In [51]:
nTravelForState[nTravelForState['state']=='PA']

,state,nTravel
34,PA,316


In [52]:
#take the 2 values
PAerr = nTravelForState[nTravelForState['state'] == '17325']['nTravel']
PAcorr = nTravelForState[nTravelForState['state'] == 'PA']['nTravel']

In [53]:
#delete the wrong row
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == '17325']).reset_index()

In [54]:
nTravelForState.head()

,index,state,nTravel
0,1,AL,75
1,2,AR,10
2,3,AZ,70
3,4,CA,28
4,5,CO,103


In [55]:
#Add the right value
nTravelForState.loc[nTravelForState['state']=='PA', 'nTravel'] = sum(PAcorr, PAerr).values
nTravelForState[nTravelForState['state'] == 'PA']

,index,state,nTravel
33,34,PA,326


#### Country that have made trips become 43

### Table State and Popolation

In [56]:
nstest[1] = nstest[1].str.replace('.','')

/var/folders/h7/63fpfjks70z8hgf9c6f28cl00000gn/T/ipykernel_2266/2156957115.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nstest[1] = nstest[1].str.replace('.','')


In [57]:
state = nstest.rename(columns = {0:'state', 1:'nPopulation'})
state.head()

,state,nPopulation
0,Alabama,5024279
1,Alaska,733391
2,Arizona,7151502
3,Arkansas,3011524
4,California,39538223


#### Add state code

In [58]:
def findCountryAlpha2 (country_name):
    try:
        sub = pycountry.subdivisions.lookup(country_name)
        sample_str = sub.code
        stateCode = sub.code[-2:]
        return stateCode
    except:
        return ("not founded!")

state['state_code'] = state.apply(lambda row: findCountryAlpha2(row.state) , axis = 1)
state.head()

,state,nPopulation,state_code
0,Alabama,5024279,AL
1,Alaska,733391,AK
2,Arizona,7151502,AZ
3,Arkansas,3011524,AR
4,California,39538223,CA


### Still errors

#### MT

In [59]:
#Library pycountry has a code wrong for Montana state
state[state['state']=='Montana']

,state,nPopulation,state_code
26,Montana,1084225,12


In [60]:
state.loc[state['state']=='Montana', 'state_code'] = 'MT'
state[state['state']=='Montana']

,state,nPopulation,state_code
26,Montana,1084225,MT


#### MY

In [61]:
# another error in library pycountry referred to Maryland code state
state[state['state']=='Maryland']

,state,nPopulation,state_code
20,Maryland,6177224,MY


In [62]:
state.loc[state['state']=='Maryland', 'state_code'] = 'MD'
state[state['state']=='Maryland']

,state,nPopulation,state_code
20,Maryland,6177224,MD


#### CSV nstest

In [63]:
#there are some state_code NB which are the same state that NJ
nTravelForState[nTravelForState['state']=='NB']

,index,state,nTravel
23,24,NB,2


In [64]:
nTravelForState[nTravelForState['state']=='NJ']

,index,state,nTravel
26,27,NJ,552


In [65]:
NJerr = nTravelForState[nTravelForState['state'] == 'NB']['nTravel']
NJcorr = nTravelForState[nTravelForState['state'] == 'NJ']['nTravel']

In [66]:
#delete the wrong row
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == 'NB']).reset_index()

In [67]:
#salvo il valore in NJ
nTravelForState.loc[nTravelForState['state']=='NJ', 'nTravel'] = sum(NJcorr, NJerr).values
nTravelForState[nTravelForState['state'] == 'NJ']

,level_0,index,state,nTravel
25,26,27,NJ,554


#### Now the number of country that have done trips are 43

### Merging time

In [68]:
table_travel_for_state = pd.merge(state, nTravelForState, left_on='state_code', right_on='state')[['state_x', 'nPopulation', 'nTravel']]
table_travel_for_state.head()

,state_x,nPopulation,nTravel
0,Alabama,5024279,75
1,Arizona,7151502,70
2,Arkansas,3011524,10
3,California,39538223,28
4,Colorado,5773714,103


In [69]:
#check if sum is correct
table_travel_for_state['nTravel'].sum()

6194

### Add ratio between the number of travels and the population

In [70]:
table_travel_for_state['nPopulation'] = table_travel_for_state['nPopulation'].apply(pd.to_numeric)

In [71]:
table_travel_for_state['ratio_Travel_Population'] = table_travel_for_state['nPopulation'] / table_travel_for_state['nTravel']
table_travel_for_state.head()

,state_x,nPopulation,nTravel,ratio_Travel_Population
0,Alabama,5024279,75,6.699039e+04
1,Arizona,7151502,70,1.021643e+05
2,Arkansas,3011524,10,3.011524e+05
3,California,39538223,28,1.412079e+06
4,Colorado,5773714,103,5.605548e+04


## 10. For each dog, compute the number of days from the posted day to the day of last access.

To solve this task the difference between the column 'posted' and the column 'accessed' will be calculated.
The information obtained will be stored in a dictionary having the dog's identification code as the key and 
the days between the posted day and the last access as value.



In [72]:
diz={}
for i in range(len(dogs)):
    diz[dogs.loc[i,'id']]= -1*(dogs.loc[i, 'posted'] - dogs.loc[i, 'accessed']).days
diz

{46042150: 0,
 46042002: 0,
 46040898: 0,
 46039877: 0,
 46039306: 0,
 46039304: 0,
 46039303: 0,
 46039302: 0,
 46039301: 0,
 46038709: 0,
 46038708: 0,
 46038703: 0,
 46038700: 0,
 46038243: 0,
 46038070: 0,
 46038064: 0,
 46038065: 0,
 46038067: 0,
 46038068: 0,
 46038060: 0,
 46038062: 0,
 46038063: 0,
 46038061: 0,
 46037951: 0,
 46037918: 0,
 46037881: 0,
 46037860: 0,
 46037820: 0,
 46037762: 0,
 46037742: 0,
 46037637: 0,
 46037534: 0,
 46036459: 1,
 46035351: 1,
 46035350: 1,
 46035353: 1,
 46035346: 1,
 46035344: 1,
 46035342: 1,
 46034532: 1,
 46033962: 1,
 46032651: 1,
 46032592: 1,
 46032594: 1,
 46032595: 1,
 46032596: 1,
 46032588: 1,
 46032587: 1,
 46032589: 1,
 46032253: 1,
 46031946: 1,
 46031507: 1,
 46031797: 1,
 46031796: 1,
 46029444: 1,
 46029446: 1,
 46028152: 1,
 46027977: 1,
 46027945: 1,
 46027921: 1,
 46027872: 1,
 46027804: 1,
 46027303: 1,
 46026629: 1,
 46026616: 1,
 46026600: 1,
 46026454: 1,
 46026507: 1,
 46026395: 1,
 46026306: 1,
 46026195: 1,
 46026

## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

### 1) for version 

In [73]:
# 1) build a table with only interesting column
duplicate_table = dogs.iloc[:, [0,13,5,34]]
duplicate_table

,id,sex,breed_primary,description
0,46042150,Male,American Staffordshire Terrier,Harley is not sure how he wound up at shelter ...
1,46042002,Male,Pit Bull Terrier,6 year old Biggie has lost his home and really...
2,46040898,Male,Shepherd,Approx 2 years old.\n Did I catch your eye? I ...
3,46039877,Female,German Shepherd Dog,NaN
4,46039306,Male,Dachshund,Theo is a friendly dachshund mix who gets alon...
...,...,...,...,...
58175,44605893,Male,Border Collie,"Due to the small size of our volunteer base, w..."
58176,44457061,Female,Australian Shepherd,NaN
58177,42865848,Female,Border Collie,"Due to the small size of our volunteer base, w..."
58178,42734734,Male,Boxer,NaN


In [74]:
#Delete all row with NaN value as despription 
duplicate_table = duplicate_table[duplicate_table['description'].notna()]

In [75]:
duplicate_female = duplicate_table[duplicate_table['sex'] == 'Female']
duplicate_male = duplicate_table[duplicate_table['sex'] == 'Male']

In [76]:
#reorder row for breed-primary
duplicate_female = duplicate_female.sort_values(by = 'breed_primary').reset_index()
duplicate_male = duplicate_male.sort_values(by = 'breed_primary').reset_index()

In [77]:
def scannerDuplicate(tab):
    duplicate = {}
    for i in range(len(tab)): #all the external row
        breed1 = tab.iloc[i]['breed_primary']
        desc1 = tab.iloc[i]['description']
        identi1 = tab.iloc[i]['id']

        for j in range(i+1,len(tab)): #all the row inside breed selected
            breed2 = tab.iloc[j]['breed_primary']
            desc2 = tab.iloc[j]['description']
            identi2 = tab.iloc[j]['id']

            if breed1 == breed2: #if equals -> compare
                if SequenceMatcher(None, desc1, desc2).ratio() >= 0.9:
                    duplicate[identi1]= 'duplicate'
                    duplicate[identi2]= 'duplicate'
            else: #different breed, break j and go on with i
                break
            
            
    return duplicate

In [78]:
prova_female = duplicate_female.iloc[0:300]
scannerDuplicate(prova_female)

{45970614: 'duplicate',
 45871731: 'duplicate',
 46023964: 'duplicate',
 46023963: 'duplicate',
 42087185: 'duplicate',
 43170920: 'duplicate',
 45986905: 'duplicate',
 45986935: 'duplicate',
 44518435: 'duplicate',
 44517341: 'duplicate',
 44130324: 'duplicate',
 45932298: 'duplicate',
 45931729: 'duplicate',
 46028451: 'duplicate',
 46028452: 'duplicate',
 46028453: 'duplicate',
 45560548: 'duplicate',
 45570899: 'duplicate',
 45949697: 'duplicate',
 46021044: 'duplicate',
 45252099: 'duplicate',
 45255573: 'duplicate',
 45872595: 'duplicate',
 45897563: 'duplicate',
 45897570: 'duplicate'}

### 2) Ml version